<a href="https://colab.research.google.com/github/chowgi/GenAI-Showcase/blob/main/mongodb_as_a_toolbox_for_llamaindex_agents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MongoDB as a Toolbox for LlamaIndex Agents

This notebook demonstrates how to leverage MongoDB Atlas as a "toolbox" for LlamaIndex agents. The application showcases the integration of MongoDB's capabilities, specifically its Vector Search feature, with LlamaIndex for building intelligent agents capable of performing various tasks by calling relevant tools stored and managed within MongoDB.

**Key Features:**

*   **MongoDB as a Tool Registry:** Instead of hardcoding tool definitions within the agent, this application stores tool metadata (name, description, parameters) directly in a MongoDB collection.
*   **MongoDB Atlas Vector Search for Tool Discovery:** LlamaIndex uses the vector embeddings of tool descriptions stored in MongoDB to perform semantic searches based on user queries. This allows the agent to dynamically discover and select the most relevant tools for a given task.
*   **LlamaIndex Agent with Function Calling:** The LlamaIndex agent is configured to use the retrieved tool definitions from MongoDB to enable function calling. This means the agent can understand the user's intent and execute the appropriate Python function (tool) stored in the application.
*   **Data Storage in MongoDB:** Besides tool definitions, the application also uses separate MongoDB collections to store operational data like customer orders, return requests, and policy documents.
*   **Integration with External Services:** The tools defined and managed in MongoDB can interact with external services (e.g., fetching real-time data, processing requests) or perform operations on the data stored within MongoDB itself (e.g., looking up order details, creating return requests).

This approach provides a flexible and scalable way to manage and expand the agent's capabilities. New tools can be added to the MongoDB collection dynamically, and the agent can discover and utilize them without requiring code changes to the agent itself.

# Environment Setup and Configuration

This section covers the installation of necessary libraries, setting up API keys, and configuring the database connection to MongoDB Atlas.

### Install required libraries

This cell installs the necessary Python libraries using `uv pip install`. These libraries include:
- `pymongo`: A Python driver for MongoDB.
- `llama-index-core`: The core LlamaIndex library.
- `llama-index-llms-openai`: LlamaIndex integration with OpenAI LLMs.
- `llama-index-embeddings-voyageai`: LlamaIndex integration with VoyageAI embeddings.
- `llama-index-vector-stores-mongodb`: LlamaIndex integration with MongoDB Atlas Vector Search.
- `llama-index-readers-file`: LlamaIndex file readers.

In [ ]:
!uv pip install pymongo llama-index-core llama-index-llms-openai llama-index-embeddings-voyageai llama-index-vector-stores-mongodb llama-index-readers-file

### Get and store API keys

Get and store API keys
This cell retrieves API keys for OpenAI, MongoDB, and VoyageAI from Google Colab's user data secrets and sets them as environment variables.

Please obtain your own API keys for OpenAI, MongoDB Atlas, and VoyageAI.

OpenAI: You can get an API key from the OpenAI website.
MongoDB Atlas: Get your connection string from your MongoDB Atlas cluster.
VoyageAI: Obtain an API key from the VoyageAI website.
Once you have your keys, add them to Google Colab's user data secrets by clicking on the "🔑" icon in the left sidebar. Name the secrets OPENAI_API_KEY, MONGODB_URI, and VOYAGE_API_KEY respectively.

It also defines the GPT model to be used.

In [ ]:
import os

from google.colab import userdata

OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

MONGO_URI = userdata.get("MONGODB_URI")
os.environ["MONGO_URI"] = MONGO_URI

VOYAGE_API_KEY = userdata.get("VOYAGE_API_KEY")
os.environ["VOYAGE_API_KEY"] = VOYAGE_API_KEY

GPT_MODEL = "gpt-4o"

### Setup the database

This cell establishes a connection to the MongoDB Atlas database using the provided URI. It then defines the database name and the names of the collections that will be used in this notebook for storing tools, orders, returns, and policies. Finally, it creates client objects for each of these collections.

In [ ]:
import pymongo

# Get MongoClient
mongo_client = pymongo.MongoClient(MONGO_URI, appname="showcase.tools.mongodb_toolbox")

# Set the DB name
db_name = "retail_agent_demo"

# Set the database client
db = mongo_client[db_name]

# Set the required collection names
tools_collection_name = "tools"
orders_collection_name = "orders"
returns_collection_name = "returns"
policies_collection_name = "policies"

tools_collection = db[tools_collection_name]
orders_collection = db[orders_collection_name]
returns_collection = db[returns_collection_name]
policies_collection = db[policies_collection_name]

# Loading Demo Data

## Download and store policy documents into MongoDB Atlas vector store

This cell downloads policy documents and stores them in a MongoDB Atlas vector store. It initializes a vector store, checks if the collection is empty, downloads PDF documents, loads them, adds metadata, initializes embedding and node parsing, parses documents into nodes, creates a storage context, creates a vector index, and ingests the documents.

In [ ]:
import os

import requests
from llama_index.core import StorageContext, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.voyageai import VoyageEmbedding

# Import PDFReader
from llama_index.readers.file import PDFReader
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

# Set up vector store for the policies collection
policy_vector_store = MongoDBAtlasVectorSearch(
    mongo_client,
    db_name=db_name,
    collection_name="policies",
    vector_index_name="vector_index",  # Assuming a vector index named 'vector_index' exists
)

# Check if the policies collection is empty
policies_count = policy_vector_store.collection.count_documents({})
if policies_count > 0:
    print(
        f"Policies collection is not empty. Skipping document import. Total documents: {policies_count}"
    )
else:
    print("Policies collection is empty. Starting document import.")

    # Define the list of document URLs
    document_urls = [
        "https://mongodb-llamaindex-demos.s3.us-west-1.amazonaws.com/privacy_policy.pdf",
        "https://mongodb-llamaindex-demos.s3.us-west-1.amazonaws.com/return_policy.pdf",
        "https://mongodb-llamaindex-demos.s3.us-west-1.amazonaws.com/shipping_policy.pdf",
        "https://mongodb-llamaindex-demos.s3.us-west-1.amazonaws.com/terms_of_service.pdf",
        "https://mongodb-llamaindex-demos.s3.us-west-1.amazonaws.com/warranty_policy.pdf",
    ]

    # Create a temporary directory to store the downloaded files
    temp_dir = "temp_policy_docs"
    os.makedirs(temp_dir, exist_ok=True)

    # Download each file to the temporary directory
    local_files = []
    for url in document_urls:
        file_name = os.path.join(temp_dir, url.split("/")[-1])
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an HTTPError for bad responses
            with open(file_name, "wb") as f:
                f.write(response.content)
            local_files.append(file_name)
            print(f"Downloaded {url} to {file_name}")
        except requests.exceptions.RequestException as e:
            print(f"Error downloading {url}: {e}")

    # Use PDFReader to load each PDF file from the temporary directory
    documents = []
    for file_path in local_files:
        try:
            loader = PDFReader()
            docs = loader.load_data(file=file_path)
            documents.extend(docs)
            print(f"Loaded {file_path}")
        except Exception as e:
            print(f"Error loading {file_path} with PDFReader: {e}")

    # Add metadata to documents (optional, but can be useful)
    for i, doc in enumerate(documents):
        doc.metadata.update(
            {
                "document_type": "policy",
                "document_index": i,
                "file_name": os.path.basename(doc.metadata.get("file_path", "unknown")),
            }
        )

    print(f"Loaded {len(documents)} documents from directory")

    # Initialize embedding model
    embed_model = VoyageEmbedding(model_name="voyage-3.5-lite", api_key=VOYAGE_API_KEY)

    # Initialize node parser for chunking
    node_parser = SentenceSplitter(chunk_size=2024, chunk_overlap=200)

    # Parse documents into nodes (chunks)
    nodes = node_parser.get_nodes_from_documents(documents)
    print(f"Created {len(nodes)} text chunks")

    # Create storage context with MongoDB vector store
    storage_context = StorageContext.from_defaults(vector_store=policy_vector_store)

    # Create vector index and ingest documents into MongoDB
    # This step automatically adds nodes to the vector store when storage_context is provided
    index = VectorStoreIndex(
        nodes=nodes,
        storage_context=storage_context,
        embed_model=embed_model,
        show_progress=True,
    )

    print("Successfully ingested all PDF documents into MongoDB 'policies' collection")

    # Display collection stats using the vector store's collection object
    policies_count = policy_vector_store.collection.count_documents({})
    print(f"Total documents in 'policies' collection: {policies_count}")

    # Optional: Clean up the temporary directory
    # import shutil
    # shutil.rmtree(temp_dir)
    # print(f"Cleaned up temporary directory: {temp_dir}")

## Create and Store Dummy Order Data

This cell generates a list of fake order data with details like order ID, date, status, total amount, shipping address, payment method, and items. It then checks if the `orders` collection in MongoDB is empty. If it is, the fake order data is inserted into the `orders` collection. This is done to populate the database with sample data for testing and demonstrating the order lookup functionality later in the notebook.

In [ ]:
from datetime import datetime

# Check if the collection is empty
if orders_collection.count_documents({}) == 0:
    # Define some fake order data
    fake_orders = [
        {
            "order_id": 101,
            "order_date": datetime(2023, 10, 26, 10, 0, 0),
            "status": "Shipped",
            "total_amount": 150.75,
            "shipping_address": "123 Main St, Anytown, CA 91234",
            "payment_method": "Credit Card",
            "items": [
                {"name": "Laptop", "price": 1200.00},
                {"name": "Mouse", "price": 25.75},
            ],
        },
        {
            "order_id": 102,
            "order_date": datetime(2023, 10, 25, 14, 30, 0),
            "status": "Processing",
            "total_amount": 55.00,
            "shipping_address": "456 Oak Ave, Somewhere, NY 54321",
            "payment_method": "PayPal",
            "items": [
                {"name": "Keyboard", "price": 75.00},
            ],
        },
        {
            "order_id": 103,
            "order_date": datetime(2023, 10, 25, 14, 30, 0),
            "status": "Processing",
            "total_amount": 35.00,
            "shipping_address": "789 Pine Rd, Elsewhere, TX 67890",
            "payment_method": "Debit Card",
            "items": [
                {"name": "Monitor", "price": 250.00},
            ],
        },
    ]

    # Insert the fake orders into the collection
    orders_collection.insert_many(fake_orders)
    print(f"Inserted {len(fake_orders)} fake orders.")
else:
    print("Orders collection is not empty. Skipping insertion of fake orders.")

# Application Setup and Configuration

## Define MongoDB Tool Decorator

This cell defines the `mongodb_toolbox` decorator. This decorator is used to register functions as tools that can be discovered and used by the LlamaIndex agent. It also handles generating embeddings for the tool descriptions and storing them in the MongoDB 'tools' collection for vector search.

In [ ]:
import inspect
from functools import wraps
from typing import get_type_hints

from llama_index.core import Document, StorageContext, VectorStoreIndex
from llama_index.core.node_parser import SentenceSplitter
from llama_index.embeddings.voyageai import VoyageEmbedding
from llama_index.vector_stores.mongodb import MongoDBAtlasVectorSearch

# Initialize vector store
vector_store = MongoDBAtlasVectorSearch(
    mongo_client,
    db_name=db_name,
    collection_name="tools",
    vector_index_name="vector_index",
)

# Initialize VoyageAIEmbedding
voyage_embed_model = VoyageEmbedding(
    model_name="voyage-3.5-lite",
)

# Create a registry for decorated tools
decorated_tools_registry = {}


def get_embedding(text):
    text = text.replace("\n", " ")
    return voyage_embed_model.get_text_embedding(text)


def mongodb_toolbox(vector_store=None):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            return func(*args, **kwargs)

        # Generate tool definition
        signature = inspect.signature(func)
        docstring = inspect.getdoc(func) or ""
        type_hints = get_type_hints(func)

        tool_def = {
            "name": func.__name__,
            "description": docstring.strip(),
            "parameters": {"type": "object", "properties": {}, "required": []},
        }

        for param_name, param in signature.parameters.items():
            if (
                param.kind == inspect.Parameter.VAR_POSITIONAL
                or param.kind == inspect.Parameter.VAR_KEYWORD
            ):
                continue

            param_type = type_hints.get(param_name, type(None))
            json_type = "string"  # Default to string
            if param_type in (int, float):
                json_type = "number"
            elif param_type is bool:
                json_type = "boolean"

            tool_def["parameters"]["properties"][param_name] = {
                "type": json_type,
                "description": f"Parameter {param_name}",
            }

            if param.default == inspect.Parameter.empty:
                tool_def["parameters"]["required"].append(param_name)

        tool_def["parameters"]["additionalProperties"] = False

        # Create Document for vector storage with embedding
        document = Document(text=tool_def["description"], metadata=tool_def)

        # Generate and set the embedding
        if vector_store and tool_def["description"]:
            embedding = voyage_embed_model.get_text_embedding(tool_def["description"])
            document.embedding = embedding

        # Add to vector store only if a document with the same name does not exist
        if vector_store:
            existing_doc = vector_store.collection.find_one(
                {"metadata.name": tool_def["name"]}
            )
            if not existing_doc:
                vector_store.add([document])
            else:
                print(
                    f"Document for tool '{tool_def['name']}' already exists. Skipping insertion."
                )

        # Register the decorated function
        decorated_tools_registry[func.__name__] = func

        return wrapper

    return decorator

## Setup indexes

This cell checks for and creates vector search indexes on the specified MongoDB collections if they don't already exist. These indexes are crucial for performing efficient vector searches on the data stored in these collections.

In [ ]:
import time

# Require vector index list
required_indexs = [
    orders_collection_name,
    tools_collection_name,
    returns_collection_name,
    policies_collection_name,
]

# Flag to track if any index was created
index_created = False

for collection_name in required_indexs:
    print(f"Checking and creating index for collection: {collection_name}")
    # Set up vector store for the current collection
    current_vector_store = MongoDBAtlasVectorSearch(
        mongo_client,
        db_name=db_name,
        collection_name=collection_name,
        vector_index_name="vector_index",
    )

    # Check if vector index exists
    try:
        search_indexes = list(current_vector_store.collection.list_search_indexes())
        index_exists = any(
            index.get("name") == "vector_index" for index in search_indexes
        )
    except Exception as e:
        print(f"Could not check search indexes for {collection_name}: {e}")
        index_exists = False

    if not index_exists:
        # Index does not exist, create it
        current_vector_store.create_vector_search_index(
            dimensions=1024, path="embedding", similarity="cosine"
        )
        print(f"Vector search index created successfully for {collection_name}.")
        index_created = True  # Set flag if an index was created
    else:
        # Index exists, skip creation
        print(
            f"Vector search index already exists for {collection_name}. Skipping creation."
        )

# Add a single 20-second pause after checking all collections, only if an index was created
if index_created:
    print("Pausing for 20 seconds to allow index builds...")
    time.sleep(20)
    print("Resuming after pause.")
else:
    print("No new indexes were created. Skipping pause.")

## Define Vector Search Function

This cell defines the `vector_search_tools` function, which performs a vector search on a given LlamaIndex vector store based on a user query. It uses the specified vector store and embedding model to find the most relevant documents (in this case, tool definitions) and returns a list of their metadata.

In [ ]:
def vector_search_tools(user_query, vector_store, top_k=3):
    """
    Perform a vector search using LlamaIndex vector store.

    Args:
        user_query (str): The user's query string.
        vector_store: The LlamaIndex vector store instance.
        top_k (int): Number of top results to return.

    Returns:
        list: A list of matching tool definitions.
    """
    # Create index from vector store
    index = VectorStoreIndex.from_vector_store(
        vector_store,
        embed_model=voyage_embed_model,
    )

    # Create query engine
    query_engine = index.as_query_engine(similarity_top_k=top_k)

    # Perform query
    response = query_engine.query(user_query)

    # Extract tool definitions from source nodes
    tools_data = []
    for node in response.source_nodes:
        tool_metadata = node.node.metadata
        tools_data.append(tool_metadata)

    return tools_data

## Define MongoDB Tools

This cell defines several Python functions that will serve as tools for the LlamaIndex agent. Each function is decorated with the `@mongodb_toolbox` decorator, which registers the function and stores its definition and embedding in the 'tools' collection in MongoDB. These tools include functions for shouting, getting weather, getting stock price, getting current time, looking up orders, responding in Spanish, checking return policy, and creating a return request.

In [ ]:
import random
from datetime import datetime


@mongodb_toolbox(vector_store=vector_store)
def get_current_time(timezone: str = "UTC") -> str:
    """
    Get the current time for a specified timezone.
    Use this when a user asks about the current time in a specific timezone.

    :param timezone: The timezone to get the current time for. Defaults to 'UTC'.
    :return: A string with the current time in the specified timezone.
    """
    current_time = datetime.utcnow().strftime("%H:%M:%S")
    return f"The current time in {timezone} is {current_time}."


@mongodb_toolbox(vector_store=vector_store)
def lookup_order_number(order_id: int) -> str:
    """
    Lookup the details of a specific order number using its order ID.
    Use this when a user asks for information about a particular order.

    :param order_id: The unique identifier of the order to look up.
    :return: A string containing the order details or a message if the order is not found.
    """
    # Get the orders collection (assuming 'db' is accessible from here)
    orders_collection = db["orders"]

    # Find the order by order_id
    order = orders_collection.find_one(
        {"order_id": order_id}
    )  # Note: Sample data uses 'order_id'

    if order:
        # Format the order details into a readable string
        order_details = f"Order ID: {order.get('order_id')}\n"
        order_details += (
            f"Order Date: {order.get('order_date').strftime('%Y-%m-%d %H:%M:%S')}\n"
        )
        order_details += f"Status: {order.get('status')}\n"
        order_details += f"Total Amount: ${order.get('total_amount')}\n"
        order_details += f"Shipping Address: {order.get('shipping_address')}\n"
        order_details += f"Payment Method: {order.get('payment_method')}\n"
        order_details += "Items:\n"
        for item in order.get("items", []):
            order_details += f"- {item.get('name')}: ${item.get('price')}\n"

        return order_details
    else:
        return f"Order with ID {order_id} not found."


@mongodb_toolbox(vector_store=vector_store)
def return_policy(return_request_description: str) -> str:
    """
    Performs search on the policies collection to determine if a user's
    return request aligns with the company's return policy, warranty policy,
    or terms of service. Use this tool when a user asks about returning an item
    and you need to check the relevant company policies.

    Args:
        return_request_description (str): A detailed description of the user's
                                          return request, including reasons
                                          for return, item condition, and any
                                          relevant order information.

    Returns:
        str: A string containing relevant policy information found through
             vector search that can help determine if the return request
             meets the company's policy.
    """
    # Create index from the policy vector store
    policy_index = VectorStoreIndex.from_vector_store(
        policy_vector_store,
        embed_model=voyage_embed_model,
    )

    # Create query engine for the policy index
    policy_query_engine = policy_index.as_query_engine(
        similarity_top_k=3
    )  # Adjust top_k as needed

    # Perform query on the policies collection with the user's return request
    response = policy_query_engine.query(return_request_description)

    # Return the response text from the query engine
    return str(response)


@mongodb_toolbox(vector_store=vector_store)
def create_return_request(order_id: int, reason: str) -> str:
    """
    Creates a return request entry for a given order ID with the specified reason.
    Use this when a user wants to initiate a return for an item from a specific order.

    :param order_id: The unique identifier of the order for which the return is requested.
    :param reason: The reason for the return.
    :return: A string confirming the return creation or indicating if the order was not found.
    """
    # Get the orders and returns collections
    orders_collection = db["orders"]
    returns_collection = db["returns"]

    # Find the order by order_id
    order = orders_collection.find_one({"order_id": order_id})

    if order:
        # Create a return document
        return_data = {
            "return_id": returns_collection.count_documents({})
            + 1,  # Simple auto-incrementing ID
            "order_id": order_id,
            "return_date": datetime.utcnow(),
            "reason": reason,
            "status": "Pending",  # Initial status
            "items": order.get("items", []),  # Include items from the original order
        }

        # Insert the return document into the returns collection
        returns_collection.insert_one(return_data)

        return f"Return request created successfully for order {order_id} with reason: {reason}."
    else:
        return f"Order with ID {order_id} not found. Could not create return."


@mongodb_toolbox(vector_store=vector_store)
def greet_user(name: str) -> str:
    """
    Greets the user by name.
    Use this when a user provides their name and you want to greet them.

    :param name: The name of the user.
    :return: A greeting message.
    """
    return f"Hello, {name}! Nice to meet you."


@mongodb_toolbox(vector_store=vector_store)
def calculate_square_root(number: float) -> str:
    """
    Calculates the square root of a given number.
    Use this when a user asks for the square root of a number.

    :param number: The number for which to calculate the square root.
    :return: A string with the square root result.
    """
    if number < 0:
        return "Cannot calculate the square root of a negative number."
    return f"The square root of {number} is {number**0.5}."


@mongodb_toolbox(vector_store=vector_store)
def repeat_phrase(phrase: str, times: int = 1) -> str:
    """
    Repeats a given phrase a specified number of times.
    Use this when a user asks you to repeat something.

    :param phrase: The phrase to repeat.
    :param times: The number of times to repeat the phrase. Defaults to 1.
    :return: A string with the repeated phrase.
    """
    if times <= 0:
        return "Please specify a positive number of times to repeat."
    return (phrase + " ") * times


@mongodb_toolbox(vector_store=vector_store)
def roll_dice(number_of_dice: int = 1, sides: int = 6) -> str:
    """
    Rolls a specified number of dice with a given number of sides and returns the results.
    Use this when a user asks to roll dice.

    :param number_of_dice: The number of dice to roll. Defaults to 1.
    :param sides: The number of sides on each die. Defaults to 6.
    :return: A string showing the result of each roll and the total.
    """
    if number_of_dice <= 0 or sides <= 0:
        return "Please specify a positive number of dice and sides."
    rolls = [random.randint(1, sides) for _ in range(number_of_dice)]
    total = sum(rolls)
    return f"You rolled {number_of_dice} dice with {sides} sides each. Results: {rolls}. Total: {total}."


@mongodb_toolbox(vector_store=vector_store)
def flip_coin(number_of_flips: int = 1) -> str:
    """
    Flips a coin a specified number of times and returns the results.
    Use this when a user asks to flip a coin.

    :param number_of_flips: The number of times to flip the coin. Defaults to 1.
    :return: A string showing the result of each flip.
    """
    if number_of_flips <= 0:
        return "Please specify a positive number of flips."
    results = [random.choice(["Heads", "Tails"]) for _ in range(number_of_flips)]
    return f"You flipped the coin {number_of_flips} times. Results: {results}."


@mongodb_toolbox(vector_store=vector_store)
def generate_random_password(length: int = 12) -> str:
    """
    Generates a random password of a specified length.
    Use this when a user asks for a random password.

    :param length: The desired length of the password. Defaults to 12.
    :return: A randomly generated password string.
    """
    if length <= 0:
        return "Please specify a positive password length."
    import string

    characters = string.ascii_letters + string.digits + string.punctuation
    password = "".join(random.choice(characters) for i in range(length))
    return f"Here is a random password: {password}"

## Populate Tools Function

This cell defines the `populate_tools` function. This function takes the results from a vector search (which are tool definitions) and converts them into a list of LlamaIndex `FunctionTool` objects. It looks up the actual function object in the `decorated_tools_registry` based on the tool name found in the search results and creates a `FunctionTool` with the corresponding function and its description.

In [ ]:
from llama_index.core.tools import FunctionTool

# Access the registry created in the mongodb_toolbox decorator definition cell
from __main__ import decorated_tools_registry


def populate_tools(search_results):
    """
    Populate the tools array based on the results from the vector search,
    returning LlamaIndex FunctionTool objects.

    Args:
    search_results (list): The list of documents returned from the vector search.

    Returns:
    list: A list of LlamaIndex FunctionTool objects.
    """
    tools = []

    for result in search_results:
        tool_name = result["name"]
        # Look up the function object in the registry
        function_obj = decorated_tools_registry.get(tool_name)

        if function_obj:
            # Create a FunctionTool object from the actual function and its description
            description = result.get("description", function_obj.__doc__ or "")
            tools.append(
                FunctionTool.from_defaults(fn=function_obj, description=description)
            )
    return tools

# Running the Agent

## Test Tool Retrieval

This cell demonstrates how to use the `vector_search_tools` function to find relevant tools based on a user query and then uses the `populate_tools` function to convert the search results into LlamaIndex `FunctionTool` objects. Finally, it prints the names of the retrieved tools to verify the process.

In [ ]:
import pprint

user_query = (
    "I want to return a damaged laptop from order 101. What is the return policy??"
)

tools_related_to_user_query = vector_search_tools(user_query, vector_store)

# populate_tools now returns FunctionTool objects
tools = populate_tools(tools_related_to_user_query)


# Iterate through the list of FunctionTool objects and print their names
tool_names = [tool.metadata.name for tool in tools]
print(
    "Selected tools from the toolbox based on the similarity to the users intention -"
)
pprint.pprint(tool_names)

In [ ]:
from llama_index.core.agent.workflow import FunctionAgent
from llama_index.core.memory import Memory
from llama_index.llms.openai import OpenAI

# Setup The agent
llm = OpenAI(model=GPT_MODEL)

tools = populate_tools(tools_related_to_user_query)

memory = Memory.from_defaults(session_id="my_session", token_limit=40000)

agent = FunctionAgent(llm=llm, tools=tools)

# Get the answer
response = await agent.run("How much did I pay for order 101", memory=memory)
print(response)

In [ ]:
response = await agent.run(
    "I want to return a damaged laptop from order 101. What is the return policy??",
    memory=memory,
)
print(response)

In [ ]:
response = await agent.run("Yes please", memory=memory)
print(response)

### Get and store API keys

This cell retrieves API keys for OpenAI, MongoDB, and VoyageAI from Google Colab's user data secrets and sets them as environment variables.

**Please obtain your own API keys for OpenAI, MongoDB Atlas, and VoyageAI.**

*   **OpenAI:** You can get an API key from the [OpenAI website](https://platform.openai.com/).
*   **MongoDB Atlas:** Get your connection string from your MongoDB Atlas cluster.
*   **VoyageAI:** Obtain an API key from the [VoyageAI website](https://voyageai.com/).

Once you have your keys, add them to Google Colab's user data secrets by clicking on the "🔑" icon in the left sidebar. Name the secrets `OPENAI_API_KEY`, `MONGODB_URI`, and `VOYAGE_API_KEY` respectively.

It also defines the GPT model to be used.